## Programa 01

Nesta aula iremos criar um algoritmo em python que pegue os dados de uma API e insira em uma tabela do postgres criado no primeiro passo

## Importar as libs da aula

In [44]:
import pandas as pd
import requests
import io
from sqlalchemy import create_engine

## Requisição API

In [45]:
url = 'https://www.harve.com.br/praticas/titanic-pt-BR.csv'
response = requests.get(url)

'''
Geralmente e necessario algum nivel de autenticacao do usuario para conseguir os dados de um endpoint(url).
Com uma chave de acesso ou token ou outra forma d autenticacao, o metodo get() ou post() do metodo requests, 
deve receber nas variaveis headers.

headers = {'Authorization': 'Bearer <YOUR_TOKEN>'}
response = requests.get(url_req,headers=headers)
'''

"\nGeralmente e necessario algum nivel de autenticacao do usuario para conseguir os dados de um endpoint(url).\nCom uma chave de acesso ou token ou outra forma d autenticacao, o metodo get() ou post() do metodo requests, \ndeve receber nas variaveis headers.\n\nheaders = {'Authorization': 'Bearer <YOUR_TOKEN>'}\nresponse = requests.get(url_req,headers=headers)\n"

## Transformando Resposta da API em Dataframe

In [46]:
'''
Utilizamos a funcao io.StringIO pois o primeiro parametro do metodo pd.read_csv pode ser o caminho para o arquivo ou
os dados em si. Mas se e passado uma string para o primeiro paramentro de read_csv, ele vai entender que vc esta passando
o caminho para o arquivo e dara problema pois ele nao encontrara este arquivo.

Com isso a funcao io.StringIO transformara os dados em um objeto para que o pandas entenda que estara mexendo com
os dados em si e nao com o caminho do arquivo.
'''
buffer = io.StringIO(response.text)
df = pd.read_csv(buffer)

## Criando a tabela no banco de dados
Para criar uma tabela no banco de dados precisaremos saber exatamente como serao as colunas e onde isto sera inserido. No caso utilizaremos o banco Harve no schema public e a tabela tera o nome titanic

In [47]:
''' 
Esta celula de codigo pegara cada coluna do csv, mudara cada espaco por underline e tirara qualquer 
espaco antes ou depois da coluna. Por exemplo:
Antes: Id do passageiro
Depois: Id_do_passageiro
'''
colunas = [coluna.strip().replace(' ','_') for coluna in df.columns]
print(colunas)

# TODO: Como voce poderia alterar o nome de cada coluna no dataframe? 
# renameColumns = {column:column.strip().replace(' ','_') for column in df.columns}
# df.rename(columns=renameColumns,inplace=True)

['Id_do_passageiro', 'Sobrevivente', 'Classe', 'Nome', 'Sexo', 'Idade', 'Irmaos_ou_conjuge', 'Pais_ou_filhos', 'Ticket', 'Tarifa_do_passageiro', 'Cabine', 'Local_de_embarque']


## Criando tabela no DW
Para criar a tabela no banco de dados podemos usar o comando abaixo, colando-o diretamento no Dbeaver ou outro DBMS.

SQL:

create table public.titanic (
Id_do_passageiro text,
Sobrevivente text,
Classe text,
Nome text,
Sexo text,
Idade text,
Irmaos_ou_conjuge text,
Pais_ou_filhos text,
Ticket text,
Tarifa_do_passageiro text,
Cabine text,
Local_de_embarque text
)

## Credenciais e Conexao com o banco

In [48]:
def connect_database(db, user, host, port, passwd):
    """
    Get SQLalchemy engine using credentials.
    Input:
    db: database name
    user: Username
    host: Hostname of the database server
    port: Port number
    passwd: Password for the database
    """

    url = f'postgresql://{user}:{passwd}@{host}:{port}/{db}'
    engine = create_engine(url)
    con = engine.connect()
    return engine, con

In [49]:
engine, con = connect_database(
                                db='harve',
                                user='<usuario>',
                                passwd='<senha>',
                                port='5432'
                            )

## Inserindo os dados no Banco

In [50]:
con.execute('''truncate table public.titanic''')
df.to_sql( 
    name='titanic',
    con=engine,
    schema='public',
    if_exists='append',
    index=False
)